In [1]:
import os
import time
import math
import json
import logging
import datetime
import yfinance as yf
from datetime import datetime, timedelta
from typing import List, Dict
import torch.nn.functional as F
import requests
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from newspaper import Article, Config
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

### Configuration:

In [2]:
QUERY = "Bitcoin"
YEARS_BACK = 2
GDELT_BASE = "https://api.gdeltproject.org/api/v2/doc/doc"
OUT_PARQUET = f"gdelt_{QUERY}_2yrs_finbert.parquet"
OUT_WEEKLY_PNG = f"gdelt_{QUERY}_2yrs_weekly_sentiment.png"
MODEL_NAME = "yiyanghkust/finbert-tone"
MAX_PER_CALL = 250
PAUSE_BETWEEN_CALLS = 5  # FIXED: GDELT requires 5 seconds minimum
PAUSE_BETWEEN_DOWNLOADS = 0.5
MAX_ARTICLE_CHUNKS = 6
CHUNK_CHARS = 1000
END_DATE = datetime.utcnow()
START_DATE = END_DATE - timedelta(days=365 * YEARS_BACK)
TEMP_META = 'gdelt_query_urls.jsonl'
RANDOM_STATE = 42
ARTICLES_PER_WEEK = 10

print('Config set. Query:', QUERY)

Config set. Query: Bitcoin


### Functions for GDELT querying:

In [3]:
def gdelt_datefmt(dt: datetime) -> str:
    """Convert datetime to GDELT format."""
    return dt.strftime('%Y%m%d%H%M%S')


def query_gdelt(query: str, start_dt: datetime, end_dt: datetime, maxrecords: int = MAX_PER_CALL):
    """Query GDELT API for articles."""
    params = {
        'query': query,
        'mode': 'artlist',
        'maxrecords': maxrecords,
        'format': 'json',
        'startdatetime': gdelt_datefmt(start_dt),
        'enddatetime': gdelt_datefmt(end_dt),
        'sort': 'datedesc'
    }
    try:
        resp = requests.get(GDELT_BASE, params=params, timeout=60)
        if resp.status_code != 200:
            logging.warning('GDELT query failed: %s -> %s', resp.status_code, resp.text[:200])
            return None
        return resp.json()
    except requests.exceptions.RequestException as e:
        logging.error(f'GDELT request exception: {e}')
        return None


def query_gdelt_with_retry(query: str, start_dt: datetime, end_dt: datetime,
                           maxrecords: int = MAX_PER_CALL, max_retries: int = 3):
    """Query GDELT with retry logic."""
    for attempt in range(max_retries):
        result = query_gdelt(query, start_dt, end_dt, maxrecords)
        if result is not None:
            return result
        if attempt < max_retries - 1:
            wait_time = (attempt + 1) * 5
            logging.warning(f'Retrying GDELT query in {wait_time}s (attempt {attempt+1}/{max_retries})')
            time.sleep(wait_time)
    logging.error(f'GDELT query failed after {max_retries} attempts')
    return None

In [4]:
# Helper function to iterate month windows

def iterate_month_windows(start: datetime, end: datetime):
    """Generate month-by-month windows between start and end dates."""
    cur = datetime(start.year, start.month, 1)
    while cur < end:
        if cur.month == 12:
            nxt = datetime(cur.year + 1, 1, 1)
        else:
            nxt = datetime(cur.year, cur.month + 1, 1)
        yield cur, min(nxt - timedelta(seconds=1), end)
        cur = nxt

In [5]:
# Main function to fetch all GDELT URLs with resume capability 

def fetch_all_gdelt_urls(query: str, start: datetime, end: datetime, out_meta_file: str = TEMP_META):
    """Query GDELT month-by-month and write metadata as JSONL for resume capability."""
    seen_urls = set()

    # Resume if meta file exists
    if os.path.exists(out_meta_file):
        logging.info("Resuming: loading existing metadata to avoid re-fetching")
        with open(out_meta_file, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    o = json.loads(line)
                    seen_urls.add(o.get("url"))
                except (json.JSONDecodeError, KeyError) as e:
                    logging.debug(f"Skipping malformed line: {e}")
                    continue

    with open(out_meta_file, "a", encoding="utf-8") as out:
        for wstart, wend in iterate_month_windows(start, end):
            logging.info("Querying GDELT for %s -> %s", wstart.date(), wend.date())
            resp = query_gdelt_with_retry(query, wstart, wend)
            time.sleep(PAUSE_BETWEEN_CALLS)

            if not resp:
                continue

            articles = resp.get("articles") or resp.get("artlist") or []
            logging.info("Found %d articles in this window", len(articles))

            for a in articles:
                url = a.get("url") or a.get("sourceurl") or a.get("documentidentifier")
                title = a.get("title") or a.get("urltitle") or ""
                seendate = a.get("seendate") or a.get("date") or a.get("published")

                if not url or url in seen_urls:
                    continue

                seen_urls.add(url)
                record = {"url": url, "title": title, "seendate": seendate}
                out.write(json.dumps(record, ensure_ascii=False) + "\n")

    logging.info("GDELT URL fetching complete. Total unique URLs: %d", len(seen_urls))
    return out_meta_file

In [6]:
# Function to load URLs from metadata file

def load_urls_from_meta(meta_file: str):
    """Load article metadata from JSONL file."""
    rows = []
    with open(meta_file, "r", encoding="utf-8") as f:
        for line in f:
            try:
                rows.append(json.loads(line))
            except (json.JSONDecodeError, ValueError) as e:
                logging.debug(f"Skipping invalid JSON line: {e}")
                continue
    return rows

In [7]:
# Function to extract text using newspaper3k 

def extract_text_with_newspaper(url: str, fallback_text: str = ""):
    """Extract article text using newspaper3k with proper timeout configuration."""
    try:
        config = Config()
        config.browser_user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        config.request_timeout = 15
        config.number_threads = 1

        art = Article(url, config=config, language='en')
        art.download()
        art.parse()
        return art.title or None, art.text or None, (art.publish_date if hasattr(art, "publish_date") else None)
    except Exception as e:
        logging.debug(f"Article extraction failed for {url}: {type(e).__name__}")
        return None, fallback_text or None, None


def chunk_text(text: str, max_chars=CHUNK_CHARS):
    """Split text into chunks based on character limit."""
    if not text:
        return []
    text = text.strip()
    if len(text) <= max_chars:
        return [text]

    sentences = text.split('. ')
    chunks = []
    cur, cur_len = [], 0

    for s in sentences:
        s_len = len(s) + 2
        if cur_len + s_len > max_chars and cur:
            chunks.append('. '.join(cur).strip() + ('.' if not cur[-1].endswith('.') else ''))
            cur = [s]
            cur_len = s_len
        else:
            cur.append(s)
            cur_len += s_len

    if cur:
        chunks.append('. '.join(cur).strip() + ('.' if not cur[-1].endswith('.') else ''))

    return chunks

In [8]:
#  Function to safely save DataFrame to Parquet with error handling 
def safe_to_parquet(df, path, **kwargs):
    """Safely save to parquet with ArrowKeyError handling."""
    import warnings
    try:
        df.to_parquet(path, engine='pyarrow', **kwargs)
    except Exception as e:
        if "pandas.period" in str(e) or "ArrowKeyError" in str(e):
            logging.warning("Using fastparquet due to period type conflict")
            try:
                df.to_parquet(path, engine='fastparquet', **kwargs)
            except ImportError:
                logging.error("fastparquet not installed. Run: pip install fastparquet")
                raise
        else:
            raise

def sanitize_df_for_parquet(df, convert_period_to='timestamp'):
    """Enhanced version with warning suppression and Period detection."""
    import warnings
    df = df.copy()

    # Handle PeriodIndex
    if isinstance(df.index, pd.PeriodIndex):
        df.index = df.index.to_timestamp() if convert_period_to == 'timestamp' else df.index.astype(str)

    for col in df.columns:
        col_dtype = df[col].dtype
        col_lower = col.lower()

        # Date/time columns with warning suppression
        if any(token in col_lower for token in ("date", "time", "published")):
            try:
                with warnings.catch_warnings():
                    warnings.filterwarnings('ignore', message='Could not infer format')
                    parsed = pd.to_datetime(df[col], errors='coerce', utc=True)
                if parsed.notna().any():
                    df[col] = parsed
                else:
                    df[col] = df[col].astype(str)
                continue
            except:
                df[col] = df[col].astype(str)
                continue

        # Handle PeriodDtype columns
        try:
            from pandas import PeriodDtype
            if isinstance(col_dtype, PeriodDtype):
                df[col] = df[col].dt.to_timestamp() if convert_period_to == 'timestamp' else df[col].astype(str)
                continue
        except:
            pass

        # Handle object columns
        if col_dtype == object:
            sample = df[col].dropna().head(50)

            # Check for Period objects
            if not sample.empty and any(isinstance(x, pd.Period) for x in sample):
                df[col] = df[col].apply(
                    lambda p: p.to_timestamp() if isinstance(p, pd.Period) else p
                ) if convert_period_to == 'timestamp' else df[col].astype(str)
                continue

            # Auto-detect datetime strings with warning suppression
            str_sample = sample.astype(str).str.strip().head(20)
            if len(str_sample) > 0:
                with warnings.catch_warnings():
                    warnings.filterwarnings('ignore', message='Could not infer format')
                    parsed_sample = pd.to_datetime(str_sample, errors='coerce', utc=True)

                if parsed_sample.notna().sum() / max(1, len(str_sample)) > 0.4:
                    try:
                        with warnings.catch_warnings():
                            warnings.filterwarnings('ignore', message='Could not infer format')
                            df[col] = pd.to_datetime(df[col], errors='coerce', utc=True)
                        continue
                    except:
                        pass

            df[col] = df[col].astype(str)

    return df

In [9]:
# Function to load FinBERT model and tokenizer 

def load_finbert_model(model_name: str = MODEL_NAME):
    """Load FinBERT tokenizer and model."""
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    logging.info(f"Loading FinBERT model on {device}")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    model.to(device)
    model.eval()

    return tokenizer, model, device


def predict_article_sentiment(tokenizer, model, device, title, text,
                              chunk_chars=CHUNK_CHARS, max_chunks=MAX_ARTICLE_CHUNKS,
                              batch_size=8, max_length=512):
    """
    Predict sentiment for an article using FinBERT.
    Returns dict with avg_probs, final_label, and n_pieces.
    """
    # Build pieces (title + text chunks)
    pieces = []
    if title:
        pieces.append(title)
    if text:
        chunks = chunk_text(text, chunk_chars)
        pieces.extend(chunks)

    if not pieces:
        return None

    pieces = pieces[:max_chunks]

    # Batch process
    all_probs = []
    label_names = None

    try:
        for i in range(0, len(pieces), batch_size):
            batch_texts = pieces[i:i+batch_size]
            inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True,
                             padding=True, max_length=max_length)
            inputs = {k: v.to(device) for k, v in inputs.items()}

            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                probs = F.softmax(logits, dim=-1).cpu().numpy()

            # Get label names
            if label_names is None:
                try:
                    id2lab = model.config.id2label
                    label_names = [id2lab[i].lower() for i in range(len(id2lab))]
                except Exception:
                    label_names = ['positive', 'neutral', 'negative'] if probs.shape[1] == 3 else [f"lab_{j}" for j in range(probs.shape[1])]

            # Store probabilities
            for row in probs:
                all_probs.append({label_names[j]: float(row[j]) for j in range(len(row))})

    except Exception as e:
        logging.error(f"Sentiment prediction failed: {type(e).__name__} - {e}")
        return None

    # Aggregate: mean probability per label
    agg = {}
    for d in all_probs:
        for k, v in d.items():
            agg.setdefault(k, []).append(v)

    avg_probs = {k: float(np.mean(vlist)) for k, vlist in agg.items()}

    # Ensure all expected labels exist
    for k in ["positive", "neutral", "negative"]:
        avg_probs.setdefault(k, 0.0)

    final_label = max(avg_probs.items(), key=lambda kv: kv[1])[0]

    return {"avg_probs": avg_probs, "final_label": final_label, "n_pieces": len(all_probs)}


In [10]:
# === STEP 1: Fetch GDELT article URLs ===

logging.info("=" * 60)
logging.info("STEP 1: Fetching GDELT article URLs")
logging.info("=" * 60)

meta_file = fetch_all_gdelt_urls(QUERY, START_DATE, END_DATE, TEMP_META)
print(f'Meta saved to {meta_file}')

2026-01-21 18:10:17,034 INFO ============================================================
2026-01-21 18:10:17,035 INFO STEP 1: Fetching GDELT article URLs
2026-01-21 18:10:17,035 INFO ============================================================
2026-01-21 18:10:17,036 INFO Querying GDELT for 2024-01-01 -> 2024-01-31
2026-01-21 18:10:17,720 WARNING GDELT query failed: 429 -> Please limit requests to one every 5 seconds or contact kalev.leetaru5@gmail.com for larger queries.


2026-01-21 18:10:17,723 WARNING Retrying GDELT query in 5s (attempt 1/3)
2026-01-21 18:10:29,503 INFO Found 250 articles in this window
2026-01-21 18:10:29,509 INFO Querying GDELT for 2024-02-01 -> 2024-02-29
2026-01-21 18:10:37,722 INFO Found 250 articles in this window
2026-01-21 18:10:37,730 INFO Querying GDELT for 2024-03-01 -> 2024-03-31
2026-01-21 18:10:45,226 INFO Found 250 articles in this window
2026-01-21 18:10:45,233 INFO Querying GDELT for 2024-04-01 -> 2024-04-30
2026-01-21 18:11:03,959 INFO Found 250 

Meta saved to gdelt_query_urls.jsonl


In [11]:
# === STEP 2: Load and sample metadata ===

logging.info("=" * 60)
logging.info("STEP 2: Loading and sampling metadata")
logging.info("=" * 60)

meta_rows = load_urls_from_meta(TEMP_META)
meta_df = pd.DataFrame(meta_rows)
meta_df['url'] = meta_df['url'].astype(str)
meta_df.drop_duplicates(subset=['url'], inplace=True)
print(f'Total unique URLs: {len(meta_df)}')

2026-01-21 18:20:33,957 INFO ============================================================
2026-01-21 18:20:33,959 INFO STEP 2: Loading and sampling metadata
2026-01-21 18:20:33,960 INFO ============================================================


Total unique URLs: 6250


In [12]:
# Parse seendate and drop invalid entries 

meta_df['seendate_parsed'] = pd.to_datetime(meta_df['seendate'], errors='coerce', utc=True)
meta_df = meta_df.dropna(subset=['seendate_parsed'])

# FIXED: Use to_timestamp() to avoid PeriodIndex issues
meta_df['week'] = meta_df['seendate_parsed'].dt.to_period('W-MON').dt.to_timestamp()

# Sample articles per week
sampled_df = meta_df.groupby('week').apply(
    lambda g: g.sample(n=min(len(g), ARTICLES_PER_WEEK), random_state=RANDOM_STATE)
).reset_index(drop=True)

print(f"Selected {len(sampled_df)} articles ({sampled_df['week'].nunique()} weeks)")
meta_df = sampled_df

Selected 250 articles (25 weeks)


/var/folders/qk/0049fg855_5dzk88ykpjq8_r0000gn/T/ipykernel_99343/4093820290.py:7: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  meta_df['week'] = meta_df['seendate_parsed'].dt.to_period('W-MON').dt.to_timestamp()
/var/folders/qk/0049fg855_5dzk88ykpjq8_r0000gn/T/ipykernel_99343/4093820290.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = meta_df.groupby('week').apply(


In [13]:
# Extract text and run FinBERT

logging.info("=" * 60)
logging.info("STEP 3: Extracting text and analyzing sentiment")
logging.info("=" * 60)

# Load existing results for resume
done_df = pd.DataFrame()
if os.path.exists(OUT_PARQUET):
    logging.info(f'Loading existing parquet {OUT_PARQUET} for resume')
    done_df = pd.read_parquet(OUT_PARQUET)

2026-01-21 18:21:38,708 INFO ============================================================
2026-01-21 18:21:38,710 INFO STEP 3: Extracting text and analyzing sentiment
2026-01-21 18:21:38,710 INFO ============================================================


In [14]:
# Prepare set of processed URLs for quick lookup

# FIXED: Use set for O(1) lookup instead of O(n)
processed_urls = set(done_df['url'].values) if not done_df.empty else set()

# Load FinBERT model
tokenizer, model, device = load_finbert_model(MODEL_NAME)

new_rows = []

for idx, r in tqdm(meta_df.iterrows(), total=len(meta_df), desc='Processing articles'):
    url = r['url']

    # FIXED: O(1) lookup
    if url in processed_urls:
        continue

    title_meta = r.get('title') or ''
    title, text, pubdate = extract_text_with_newspaper(url, fallback_text=title_meta)
    time.sleep(PAUSE_BETWEEN_DOWNLOADS)

    if not title and not text:
        logging.info(f'No content for url {url} (skipping)')
        continue

    pred = predict_article_sentiment(tokenizer, model, device, title or title_meta, text or '')
    if pred is None:
        continue

    avg_probs = pred['avg_probs']
    rec = {
        'url': url,
        'rss_title': title_meta,
        'title': title,
        'published_raw': pubdate if pubdate else r.get('seendate'),
        'sent_label': pred['final_label'],
        'prob_positive': avg_probs.get('positive', 0.0),
        'prob_neutral': avg_probs.get('neutral', 0.0),
        'prob_negative': avg_probs.get('negative', 0.0),
        'n_pieces': pred['n_pieces']
    }
    new_rows.append(rec)
    processed_urls.add(url)  # FIXED: Add to set

    # Save every 50 articles
    if len(new_rows) >= 50:
        logging.info(f'Checkpoint: Saving {len(new_rows)} new rows ({len(done_df) + len(new_rows)} total)')
        chunk_df = pd.DataFrame(new_rows)
        done_df = pd.concat([done_df, chunk_df], ignore_index=True) if not done_df.empty else chunk_df.copy()
        safe_df = sanitize_df_for_parquet(done_df, convert_period_to='timestamp')
        safe_to_parquet(safe_df, OUT_PARQUET, index=False)
        new_rows = []

# Final save
if new_rows:
    logging.info(f'Final save: {len(new_rows)} new rows')
    chunk_df = pd.DataFrame(new_rows)
    done_df = pd.concat([done_df, chunk_df], ignore_index=True) if not done_df.empty else chunk_df.copy()
    safe_df = sanitize_df_for_parquet(done_df, convert_period_to='timestamp')
    safe_to_parquet(safe_df, OUT_PARQUET, index=False)

logging.info(f'Finished extraction & prediction; total articles: {len(done_df)}')


2026-01-21 18:22:06,276 INFO Loading FinBERT model on cpu


In [ ]:
# Generate weekly sentiment plot
logging.info("=" * 60)
logging.info("STEP 4: Generating weekly sentiment plot")
logging.info("=" * 60)

if 'done_df' not in locals() or done_df.empty:
    if os.path.exists(OUT_PARQUET):
        done_df = pd.read_parquet(OUT_PARQUET)
    else:
        raise FileNotFoundError('No prediction parquet found')

In [ ]:
# Define x-axis range
start_date = pd.Timestamp("2023-10-20", tz='UTC')
end_date   = pd.Timestamp("2025-10-19", tz='UTC')

# Disable LaTeX rendering to suppress font warnings
mpl.rcParams['text.usetex'] = False
# Disable tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Load the model output (reuse in-memory results if available)
try:
    df = done_df.copy()
except NameError:
    df = pd.read_parquet(OUT_PARQUET)

# Ensure datetime and index
df['published'] = pd.to_datetime(df['published_raw'], errors='coerce', utc=True)
df = df.dropna(subset=['published'])
df = df.set_index('published').sort_index()

# Compute weekly sentiment averages (Monday-anchored)
weekly = df[['prob_positive', 'prob_neutral', 'prob_negative']].resample('W-MON').mean().sort_index()

btc = yf.download("BTC-USD", start="2023-10-20", end="2025-10-20", progress=False)
btc.index = pd.to_datetime(btc.index).tz_localize('UTC')

# Resample to weekly closing price (Monday)
btc_weekly = btc['Close'].resample('W-MON').mean()

# Compute weekly change in price (Δ price = current week - previous week)
btc_weekly_change = btc_weekly.diff()
btc_weekly_change = btc_weekly.pct_change()  # percentage change

fig, ax = plt.subplots(figsize=(14, 6))

# Weekly bar width and offset
bar_width = pd.Timedelta(days=2)
offset = pd.Timedelta(days=2)
x = weekly.index

# Plot sentiment grouped bars
ax.bar(x - offset, weekly['prob_positive'], width=bar_width, label='Positive', color='blue')
ax.bar(x,          weekly['prob_neutral'],  width=bar_width, label='Neutral',  color='cyan')
ax.bar(x + offset, weekly['prob_negative'], width=bar_width, label='Negative', color='red')

# Left axis: sentiment
ax.set_xlabel('Week')
ax.set_ylabel('Average sentiment probability')
ax.set_title(f"Weekly FinBERT Sentiment + BTC Weekly Fractional Price Change ({START_DATE.date()} → {END_DATE.date()})")
ax.grid(True, axis='y', linestyle='--', alpha=0.7)
ax.set_xlim([start_date, end_date])

# X-axis formatting
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

ax2 = ax.twinx()
ax2.plot(btc_weekly_change.index, btc_weekly_change, label='BTC Weekly Fractional Price Change (USD)', color='orange', linewidth=1, ls='--')
ax2.axhline(0, color='gray', linestyle='--', linewidth=1)  # zero-line
ax2.set_ylabel('BTC Weekly Change (USD)', color='black')
ax2.tick_params(axis='y', labelcolor='black')

# Combine legends
lines_1, labels_1 = ax.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax2.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left')

plt.tight_layout()
# plt.savefig(OUT_WEEKLY_PNG.replace('.png', '_btc_change_overlay.png'), dpi=150)
plt.show()
